In [1]:
%%capture install
%pip install manim
%pip install IPython --upgrade

In [2]:
from manim import *
import numpy as np

Manim Community v0.17.2

In [11]:
%%manim -qk -v WARNING SierpinskiTriangle

PI = np.pi
BASE_SIZE = 6
MAX_DEPTH = 6

DP_M_MAP = [[] for _ in range(MAX_DEPTH)]

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

class SierpinskiTriangle(Scene):

    def play_animation_by_depth(self, klass, longest, dec, reverse=False):
        iterator = DP_M_MAP
        if reverse:
            iterator = DP_M_MAP[::-1]
        for ii, depth in enumerate(iterator):
            if len(depth) == 1:
                self.play(klass(depth[0]), runtime=longest)
                continue
            
            rt = max(longest - dec*ii, 0.05)
            if reverse:
                rt = dec * (ii + 1)

            chunks = list(split(depth, 3))
            for i in range(len(chunks[0])):
                animations = [klass(chunks[x][i]) for x in range(3)]
                self.play(*animations, run_time=rt)

    def draw_triangle(self, center, size):
        side = size
        height = (np.sqrt(3)/2) * side
        bottom_left = center + (-side/2, -(height/2), 0)
        bottom_right = center + (side/2, -(height/2), 0)
        top = (0, height/2, 0)

        t = Polygon(
            bottom_left, bottom_right, top,
            fill_color=BLUE,
            fill_opacity=1.0
        )

        self.add(t)

        return t

    def recursive_triangle(self, center, depth, t):
        if depth == MAX_DEPTH:
            return

        # step one: big triangle
        # area = sqrt(3)/4 (side)^2
        # height = sqrt(3)/4 * side

        side = BASE_SIZE * ((1/2) ** depth)
        height = (np.sqrt(3)/2) * side

        # step two: middle triangle (empty out)
        new_height = height / 2
        new_side = side / 2
        p = Polygon(
            center + (new_side / 2, 0, 0),
            center + (-new_side/2, 0, 0),
            center + (0, -new_height, 0), 
            stroke_opacity=0.0, fill_opacity=1.0, fill_color=BLACK
        )
        DP_M_MAP[depth].append(p)
        # self.play(FadeIn(p), run_time=0.05)

        # Up
        self.recursive_triangle(center + (UP * new_height/2), depth+1, t)
        # Left-Down
        self.recursive_triangle(center + (DOWN * new_height/2) + (LEFT * new_side/2), depth+1, t)
        # Right-Down
        self.recursive_triangle(center + (DOWN * new_height/2) + (RIGHT * new_side/2), depth+1, t)

    def create_triangle(self, center, size):
        t = self.draw_triangle(center, size)

        self.recursive_triangle(np.array([0, 0, 0]), 0, t)

        # fade in
        self.play_animation_by_depth(Create, 1, 0.2)

        # fade out
        self.play_animation_by_depth(Uncreate, 1, 0.1, reverse=True)


    def construct(self):
        
        t = Text(
            "Sierpiński triangle",
            font_size=16
        ).shift(LEFT*3 + UP*2)
        t2 = Text(
            "A Sierpiński triangle is an equilateral\ntriangle that is recursively divided\ninto smaller triangles.",
            font_size=14
        ).shift(LEFT*3 + UP*1.5)

        self.add(t, t2)

        self.create_triangle(ORIGIN, BASE_SIZE)
        self.wait()
